In [7]:
import pandas as pd

# Nombre del archivo de salida
output_filename = "reporte_evaluacion_mmat.txt"

# Cargar el archivo CSV
try:
    df = pd.read_csv('analisisTodos.csv', delimiter=';')
except FileNotFoundError:
    print("Asegúrate de que 'analisisTodos.csv' esté en el mismo directorio que este script.")
    exit()
except Exception as e:
    print(f"Error al leer el archivo CSV: {e}")
    exit()

In [8]:
# Columnas relevantes para la evaluación de calidad (según tu solicitud)
relevant_columns = [
    'Authors', 'Year', 'Study Design', 'Intervention', 'Control/Comparison',
    'Setting', 'Cognitive Instruments', 'Affective Instruments', 'Other Instruments',
    'Quantitative Methods', 'Qualitative Methods', 'Mixed Methods',
    'Sample Size', 'Age/Grade', 'Special Characteristics'
]


In [9]:


# Asegurarse de que las columnas existan en el DataFrame
existing_relevant_columns = [col for col in relevant_columns if col in df.columns]

# Criterios MMAT 2018 (para referencia)
mmat_criteria = {
    "Screening": [
        "S1: ¿Son claras las preguntas/objetivos de investigación?",
        "S2: ¿Los datos recolectados permiten abordar esas preguntas/objetivos?"
    ],
    "Qualitative": [
        "1.1: ¿Es coherente el enfoque cualitativo con la pregunta de investigación?",
        "1.2: ¿La recolección de datos fue adecuada para abordar la pregunta de investigación?",
        "1.3: ¿Los hallazgos se derivan adecuadamente del análisis de datos (ej. se describen los métodos de análisis, se consideran datos contradictorios)?",
        "1.4: ¿Se consideró adecuadamente el contexto en el que se recolectaron los datos?",
        "1.5: ¿Se presentó de manera convincente la perspectiva de los participantes?"
    ],
    "Quantitative RCT": [
        "2.1: ¿Fue adecuada la aleatorización?",
        "2.2: ¿Se ocultó la asignación a la intervención?",
        "2.3: ¿Fueron comparables los grupos al inicio de la intervención?",
        "2.4: ¿Hubo cegamiento de los participantes, personal de intervención y/o evaluadores de resultados?",
        "2.5: ¿Estuvieron completos los datos de resultado para todos los participantes y se analizaron adecuadamente (ej. análisis por intención de tratar)?"
    ],
    "Quantitative Non-Randomized": [
        "3.1: ¿Se reclutaron los participantes de manera que se minimizara el sesgo de selección?",
        "3.2: ¿Se midieron las variables de manera apropiada (ej. válidas y fiables)?",
        "3.3: ¿Se midieron los resultados de manera apropiada (ej. válidos y fiables)?",
        "3.4: ¿Se tuvieron en cuenta los factores de confusión en el diseño y/o análisis?",
        "3.5: ¿Estuvieron completos los datos de resultado y se analizaron adecuadamente?"
    ],
    "Quantitative Descriptive": [
        "4.1: ¿La muestra es representativa de la población objetivo?",
        "4.2: ¿Fue adecuado el método de muestreo para abordar la pregunta de investigación?",
        "4.3: ¿Se midieron las variables de manera apropiada (ej. válidas y fiables)?",
        "4.4: ¿Fue aceptable la tasa de respuesta (≥60%) y se gestionó adecuadamente el sesgo de no respuesta?",
        "4.5: ¿Es apropiado el análisis estadístico para responder a la pregunta de investigación?"
    ],
    "Mixed Methods": [
        "5.1: ¿Existe una justificación adecuada para usar un diseño de métodos mixtos para responder a la pregunta de investigación?",
        "5.2: ¿Se integraron adecuadamente los diferentes componentes del estudio para responder a la pregunta de investigación (ej. los hallazgos cualitativos y cuantitativos están conectados, integrados o comparados)?",
        "5.3: ¿Se consideraron las limitaciones asociadas con la integración de los componentes cualitativos y cuantitativos?",
        "5.4: ¿Los resultados de los componentes cualitativos y cuantitativos se presentan de manera que se aborde la pregunta de investigación?",
        "5.5: ¿Se abordó la calidad de los componentes cualitativos y cuantitativos utilizando los criterios MMAT relevantes?" # Este es un recordatorio
    ]
}



In [10]:


def infer_study_design_category(row):
    """
    Infiere la categoría de diseño MMAT basada en las columnas del CSV.
    Esta es una inferencia simple y puede necesitar ajustes.
    """
    study_design_text = str(row.get('Study Design', '')).lower()
    quant_methods_text = str(row.get('Quantitative Methods', '')).lower()
    qual_methods_text = str(row.get('Qualitative Methods', '')).lower()
    mixed_methods_text = str(row.get('Mixed Methods', '')).lower()
    control_comparison_text = str(row.get('Control/Comparison', '')).lower()

    is_qual = 'qualitative' in study_design_text or 'cualitativo' in study_design_text or \
              ('entrevista' in qual_methods_text or 'observaci' in qual_methods_text or 'temático' in qual_methods_text or 'narrativ' in qual_methods_text) and \
              not ('cuantitativo' in study_design_text or 'quant' in study_design_text or 'experimental' in study_design_text or 'survey' in study_design_text or 'estadístic' in quant_methods_text)

    is_quant = 'quantitative' in study_design_text or 'cuantitativo' in study_design_text or \
               'experimental' in study_design_text or 'survey' in study_design_text or 'encuesta' in study_design_text or \
               ('estadístic' in quant_methods_text or 't-test' in quant_methods_text or 'análisis de regresión' in quant_methods_text or 'anova' in quant_methods_text)

    is_mixed = 'mixed' in study_design_text or 'mixto' in study_design_text or \
               'mixed methods' in mixed_methods_text or 'métodos mixtos' in mixed_methods_text or \
               (is_qual and is_quant)


    if is_mixed:
        return "Mixed Methods"
    elif is_qual:
        return "Qualitative"
    elif is_quant:
        if 'rct' in study_design_text or 'randomized' in study_design_text or 'aleatorizado' in study_design_text or \
           ('control group' in control_comparison_text and ('random' in study_design_text or 'aleatorio' in study_design_text)):
            return "Quantitative RCT"
        elif 'control' in control_comparison_text or 'compar' in control_comparison_text or 'cuasi-experimental' in study_design_text or 'quasi-experimental' in study_design_text :
            return "Quantitative Non-Randomized"
        elif 'descriptive' in study_design_text or 'descriptivo' in study_design_text or 'survey' in study_design_text or 'encuesta' in study_design_text or \
             ('frecuenc' in quant_methods_text or 'porcentajes' in quant_methods_text and not ('control' in control_comparison_text or 'compar' in control_comparison_text)):
            return "Quantitative Descriptive"
        else: # Default para quant si no se puede especificar más
            return "Quantitative Non-Randomized" # O podría ser descriptivo si no hay comparación
    else:
        # Si no se puede inferir claramente, se podría pedir al usuario o asignar una categoría por defecto/revisión manual
        print(f"Advertencia: No se pudo inferir claramente el diseño para el estudio: {row.get('Authors', 'Desconocido')}. Revisar manualmente.")
        # Intentar una inferencia más amplia basada en métodos
        if pd.notna(row.get('Quantitative Methods')) and pd.notna(row.get('Qualitative Methods')):
            return "Mixed Methods"
        if pd.notna(row.get('Qualitative Methods')):
            return "Qualitative"
        if pd.notna(row.get('Quantitative Methods')):
             # Podría ser Non-Randomized o Descriptive. Asumir Non-Randomized si hay intervención o comparación.
            if pd.notna(row.get('Intervention')) or pd.notna(row.get('Control/Comparison')):
                return "Quantitative Non-Randomized"
            else:
                return "Quantitative Descriptive"
        return "Unknown"



In [5]:
# --- Abrir archivo para escribir el reporte ---
try:
    with open(output_filename, 'w', encoding='utf-8') as report_file:
        report_file.write("Evaluación de Calidad Metodológica (MMAT 2018) - Guía\n\n")

        for index, row in df.iterrows():
            report_file.write(f"--- Estudio {index + 1} ---\n")
            report_file.write(f"Autores: {row.get('Authors', 'N/A')}, Año: {row.get('Year', 'N/A')}\n")

            report_file.write("\nInformación Relevante del Estudio:\n")
            for col in existing_relevant_columns:
                if col not in ['Authors', 'Year']: # Ya impresos
                     report_file.write(f"  {col}: {row.get(col, 'N/A')}\n")

            # Inferir categoría de diseño
            design_category = infer_study_design_category(row)
            report_file.write(f"\nCategoría MMAT Inferida: {design_category}\n")

            # Mostrar criterios MMAT
            report_file.write("\nPreguntas de Cribado MMAT:\n")
            for item in mmat_criteria["Screening"]:
                report_file.write(f"  - {item} (Sí/No/No se puede determinar)\n")

            if design_category != "Unknown" and design_category in mmat_criteria:
                report_file.write(f"\nCriterios Específicos MMAT para {design_category}:\n")
                for item in mmat_criteria[design_category]:
                    report_file.write(f"  - {item} (Sí/No/No se puede determinar)\n")
                if design_category == "Mixed Methods":
                    report_file.write("    (Recuerda evaluar también los componentes CUANTITATIVO y CUALITATIVO por separado con sus criterios MMAT)\n")
            else:
                report_file.write("\n  No se pudieron determinar los criterios específicos MMAT debido a categoría de diseño desconocida o no mapeada.\n")
            
            report_file.write("\n" + "="*50 + "\n\n")
        
        print(f"Reporte guardado exitosamente en '{output_filename}'")

except Exception as e:
    print(f"Ocurrió un error al generar el reporte: {e}")

Advertencia: No se pudo inferir claramente el diseño para el estudio: Gyöngyvér Molnár, Zoltán Hermann. Revisar manualmente.
Advertencia: No se pudo inferir claramente el diseño para el estudio: E Song, N M Suaib, A J Sihes, R Alwee, Z Mohd Yunos. Revisar manualmente.
Advertencia: No se pudo inferir claramente el diseño para el estudio: Meria Ultra Gusteti, Ronal Rifandi, Trysa Gustya Manda, Melani Putri. Revisar manualmente.
Reporte guardado exitosamente en 'reporte_evaluacion_mmat.txt'


In [11]:


# Columnas relevantes para la evaluación de calidad (según tu solicitud)
relevant_columns = [
    'Authors', 'Year', 'Study Design', 'Intervention', 'Control/Comparison',
    'Setting', 'Cognitive Instruments', 'Affective Instruments', 'Other Instruments',
    'Quantitative Methods', 'Qualitative Methods', 'Mixed Methods',
    'Sample Size', 'Age/Grade', 'Special Characteristics'
]

# Asegurarse de que las columnas existan en el DataFrame
existing_relevant_columns = [col for col in relevant_columns if col in df.columns]

# Criterios MMAT 2018 (para referencia)
mmat_criteria = {
    "Screening": [
        "S1: ¿Son claras las preguntas/objetivos de investigación?",
        "S2: ¿Los datos recolectados permiten abordar esas preguntas/objetivos?"
    ],
    "Qualitative": [
        "1.1: ¿Es coherente el enfoque cualitativo con la pregunta de investigación?",
        "1.2: ¿La recolección de datos fue adecuada para abordar la pregunta de investigación?",
        "1.3: ¿Los hallazgos se derivan adecuadamente del análisis de datos (ej. se describen los métodos de análisis, se consideran datos contradictorios)?",
        "1.4: ¿Se consideró adecuadamente el contexto en el que se recolectaron los datos?",
        "1.5: ¿Se presentó de manera convincente la perspectiva de los participantes?"
    ],
    "Quantitative RCT": [
        "2.1: ¿Fue adecuada la aleatorización?",
        "2.2: ¿Se ocultó la asignación a la intervención?",
        "2.3: ¿Fueron comparables los grupos al inicio de la intervención?",
        "2.4: ¿Hubo cegamiento de los participantes, personal de intervención y/o evaluadores de resultados?",
        "2.5: ¿Estuvieron completos los datos de resultado para todos los participantes y se analizaron adecuadamente (ej. análisis por intención de tratar)?"
    ],
    "Quantitative Non-Randomized": [
        "3.1: ¿Se reclutaron los participantes de manera que se minimizara el sesgo de selección?",
        "3.2: ¿Se midieron las variables de manera apropiada (ej. válidas y fiables)?",
        "3.3: ¿Se midieron los resultados de manera apropiada (ej. válidos y fiables)?",
        "3.4: ¿Se tuvieron en cuenta los factores de confusión en el diseño y/o análisis?",
        "3.5: ¿Estuvieron completos los datos de resultado y se analizaron adecuadamente?"
    ],
    "Quantitative Descriptive": [
        "4.1: ¿La muestra es representativa de la población objetivo?",
        "4.2: ¿Fue adecuado el método de muestreo para abordar la pregunta de investigación?",
        "4.3: ¿Se midieron las variables de manera apropiada (ej. válidas y fiables)?",
        "4.4: ¿Fue aceptable la tasa de respuesta (≥60%) y se gestionó adecuadamente el sesgo de no respuesta?",
        "4.5: ¿Es apropiado el análisis estadístico para responder a la pregunta de investigación?"
    ],
    "Mixed Methods": [
        "5.1: ¿Existe una justificación adecuada para usar un diseño de métodos mixtos para responder a la pregunta de investigación?",
        "5.2: ¿Se integraron adecuadamente los diferentes componentes del estudio para responder a la pregunta de investigación (ej. los hallazgos cualitativos y cuantitativos están conectados, integrados o comparados)?",
        "5.3: ¿Se consideraron las limitaciones asociadas con la integración de los componentes cualitativos y cuantitativos?",
        "5.4: ¿Los resultados de los componentes cualitativos y cuantitativos se presentan de manera que se aborde la pregunta de investigación?",
        "5.5: ¿Se abordó la calidad de los componentes cualitativos y cuantitativos utilizando los criterios MMAT relevantes?" # Este es un recordatorio
    ]
}

def infer_study_design_category(row):
    """
    Infiere la categoría de diseño MMAT basada en las columnas del CSV.
    Esta es una inferencia simple y puede necesitar ajustes.
    """
    study_design_text = str(row.get('Study Design', '')).lower()
    quant_methods_text = str(row.get('Quantitative Methods', '')).lower()
    qual_methods_text = str(row.get('Qualitative Methods', '')).lower()
    mixed_methods_text = str(row.get('Mixed Methods', '')).lower()
    control_comparison_text = str(row.get('Control/Comparison', '')).lower()
    intervention_text = str(row.get('Intervention', '')).lower()

    # Convertir a string para asegurar que no haya NaN o float
    is_qual_present_sm = pd.notna(row.get('Qualitative Methods')) and row.get('Qualitative Methods').strip() != ''
    is_quant_present_sm = pd.notna(row.get('Quantitative Methods')) and row.get('Quantitative Methods').strip() != ''
    is_mixed_present_sm = pd.notna(row.get('Mixed Methods')) and row.get('Mixed Methods').strip() != ''


    is_qual_keywords = 'qualitative' in study_design_text or 'cualitativo' in study_design_text or \
                       any(kw in qual_methods_text for kw in ['entrevista', 'observaci', 'temático', 'narrativ', 'caso', 'etnográf'])
    
    is_quant_keywords = 'quantitative' in study_design_text or 'cuantitativo' in study_design_text or \
                        'experimental' in study_design_text or 'survey' in study_design_text or 'encuesta' in study_design_text or \
                        any(kw in quant_methods_text for kw in ['estadístic', 't-test', 'análisis de regresión', 'anova', 'spss', 'prueba', 'correlaci'])

    is_mixed_keywords = 'mixed' in study_design_text or 'mixto' in study_design_text or \
                        'mixed methods' in mixed_methods_text or 'métodos mixtos' in mixed_methods_text or \
                        (is_qual_keywords and is_quant_keywords) or \
                        (is_qual_present_sm and is_quant_present_sm) or is_mixed_present_sm


    if is_mixed_keywords:
        return "Mixed Methods"
    elif is_qual_keywords and not is_quant_keywords : # Asegurar que no sea mixto por error
        return "Qualitative"
    elif is_quant_keywords and not is_qual_keywords: # Asegurar que no sea mixto por error
        is_rct = 'rct' in study_design_text or 'randomized' in study_design_text or 'aleatorizado' in study_design_text or \
                 ('control group' in control_comparison_text and any(kw in study_design_text for kw in ['random', 'aleatorio']))
        is_non_randomized = pd.notna(row.get('Control/Comparison')) or \
                            'cuasi-experimental' in study_design_text or 'quasi-experimental' in study_design_text or \
                            ('control' in control_comparison_text and not is_rct)
        is_descriptive = 'descriptive' in study_design_text or 'descriptivo' in study_design_text or \
                         'survey' in study_design_text or 'encuesta' in study_design_text or \
                         (any(kw in quant_methods_text for kw in ['frecuenc', 'porcentajes']) and not (is_rct or is_non_randomized))


        if is_rct:
            return "Quantitative RCT"
        elif is_non_randomized:
            return "Quantitative Non-Randomized"
        elif is_descriptive:
            return "Quantitative Descriptive"
        else: # Default para quant si no se puede especificar más
            if pd.notna(row.get('Intervention')) or pd.notna(row.get('Control/Comparison')):
                 return "Quantitative Non-Randomized" # Asumir si hay intervención/comparación
            return "Quantitative Descriptive" # Default general para cuantitativo
    else: # Casos más ambiguos, intentar con la presencia de métodos
        if is_qual_present_sm and is_quant_present_sm: # Si ambas columnas tienen contenido
             return "Mixed Methods"
        if is_qual_present_sm:
             return "Qualitative"
        if is_quant_present_sm:
            if pd.notna(row.get('Intervention')) or pd.notna(row.get('Control/Comparison')): # Si hay intervención o comparación
                return "Quantitative Non-Randomized"
            else: # Si no, probablemente descriptivo
                return "Quantitative Descriptive"
        
        print(f"Advertencia: No se pudo inferir el diseño para: {row.get('Authors', 'Desconocido')}. Revisar manualmente.")
        return "Unknown"

# --- Abrir archivo para escribir el reporte ---
try:
    with open(output_filename, 'w', encoding='utf-8') as report_file:
        report_file.write("Evaluación de Calidad Metodológica (MMAT 2018) - Guía\n\n")

        for index, row in df.iterrows():
            report_file.write(f"--- Estudio {index + 1} ---\n")
            report_file.write(f"Autores: {row.get('Authors', 'N/A')}, Año: {row.get('Year', 'N/A')}\n")

            report_file.write("\nInformación Relevante del Estudio:\n")
            for col in existing_relevant_columns:
                if col not in ['Authors', 'Year']: # Ya impresos
                     report_file.write(f"  {col}: {row.get(col, 'N/A')}\n")

            # Inferir categoría de diseño
            design_category = infer_study_design_category(row)
            report_file.write(f"\nCategoría MMAT Inferida: {design_category}\n")

            # Mostrar criterios MMAT
            report_file.write("\nPreguntas de Cribado MMAT:\n")
            for item in mmat_criteria["Screening"]:
                report_file.write(f"  - {item} (Sí/No/No se puede determinar)\n")

            if design_category != "Unknown" and design_category in mmat_criteria:
                report_file.write(f"\nCriterios Específicos MMAT para {design_category}:\n")
                for item in mmat_criteria[design_category]:
                    report_file.write(f"  - {item} (Sí/No/No se puede determinar)\n")
                if design_category == "Mixed Methods":
                    report_file.write("    (Recuerda evaluar también los componentes CUANTITATIVO y CUALITATIVO por separado con sus criterios MMAT)\n")
            else:
                report_file.write("\n  No se pudieron determinar los criterios específicos MMAT debido a categoría de diseño desconocida o no mapeada.\n")
            
            report_file.write("\n" + "="*50 + "\n\n")
        
        print(f"Reporte guardado exitosamente en '{output_filename}'")

except Exception as e:
    print(f"Ocurrió un error al generar el reporte: {e}")

Reporte guardado exitosamente en 'reporte_evaluacion_mmat.txt'


In [13]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
import json
import os
from matplotlib.path import Path
from matplotlib.patches import PathPatch
import matplotlib.patches as mpatches

# Set style parameters
matplotlib.rcParams['font.family'] = 'sans-serif'
matplotlib.rcParams['font.sans-serif'] = ['Arial', 'Helvetica', 'DejaVu Sans']
matplotlib.rcParams['font.size'] = 12
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['axes.titlesize'] = 16
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
plt.style.use('ggplot')

# Our data from the analysis
data = {
  "studyTypes": {
    "Mixed Methods": 18,
    "Quantitative": 11
  },
  "avgScores": {
    "Mixed Methods": 80,
    "Quantitative": 70.9
  },
  "compliancePercentages": {
    "Mixed Methods": {
      "Q1": 16.7,
      "Q2": 83.3,
      "Q3": 100,
      "Q4": 100,
      "Q5": 100
    },
    "Quantitative": {
      "Q1": 36.4,
      "Q2": 72.7,
      "Q3": 81.8,
      "Q4": 81.8,
      "Q5": 63.6
    }
  },
  "scoreDistribution": {
    "Mixed Methods": {
      "60": 3,
      "80": 12,
      "100": 3
    },
    "Quantitative": {
      "40": 2,
      "60": 2,
      "80": 6,
      "100": 1
    }
  },
  "criteriaLabels": {
    "Q1": "Study Design",
    "Q2": "Sampling/Collection",
    "Q3": "Instruments/Analysis",
    "Q4": "Comparison/Integration",
    "Q5": "Statistical Analysis"
  }
}

# Define colors
colors = {
    "Mixed Methods": "#3498db",  # Blue
    "Quantitative": "#e74c3c",   # Red
    "Other": "#95a5a6"           # Gray
}

# Create directory to save figures
if not os.path.exists('figures'):
    os.makedirs('figures')

# Figure 1: Study Distribution
fig, ax = plt.subplots(figsize=(10, 6))
study_types = list(data["studyTypes"].keys())
study_counts = list(data["studyTypes"].values())
study_colors = [colors[t] for t in study_types]

# Calculate percentages
total_studies = sum(study_counts)
percentages = [count/total_studies*100 for count in study_counts]

# Create pie chart
wedges, texts, autotexts = ax.pie(
    study_counts, 
    labels=study_types,
    autopct='%1.1f%%',
    startangle=90,
    colors=study_colors,
    explode=[0.05]*len(study_types),  # Slight explode for all slices
    shadow=True,
    wedgeprops={'edgecolor': 'white', 'linewidth': 1.5}
)

# Style the text and percentages
for text in texts:
    text.set_fontsize(14)
for autotext in autotexts:
    autotext.set_fontsize(12)
    autotext.set_fontweight('bold')
    autotext.set_color('white')

ax.set_title('Distribution of Studies by Methodology Type (n=29)', fontsize=18, pad=20)
plt.tight_layout()
plt.savefig('figures/study_distribution.png', dpi=300, bbox_inches='tight')
plt.close()

# Figure 2: Average MMAT Scores
fig, ax = plt.subplots(figsize=(10, 6))
study_types = list(data["avgScores"].keys())
avg_scores = list(data["avgScores"].values())
study_colors = [colors[t] for t in study_types]

bars = ax.bar(study_types, avg_scores, color=study_colors, width=0.6, edgecolor='white', linewidth=1.5)

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    ax.text(
        bar.get_x() + bar.get_width()/2., height + 1,
        f'{height:.1f}%', ha='center', va='bottom', fontsize=14, fontweight='bold'
    )

ax.set_ylim(0, 100)
ax.set_xlabel('Study Type', fontsize=16, labelpad=15)
ax.set_ylabel('Average MMAT Score (%)', fontsize=16, labelpad=15)
ax.set_title('Average Methodological Quality Score by Study Type', fontsize=18, pad=20)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('figures/average_scores.png', dpi=300, bbox_inches='tight')
plt.close()

# Figure 3: Criteria Compliance
fig, ax = plt.subplots(figsize=(12, 7))

# Get the data
criteria = list(data["criteriaLabels"].values())
x = np.arange(len(criteria))
width = 0.35

mixed_compliance = list(data["compliancePercentages"]["Mixed Methods"].values())
quant_compliance = list(data["compliancePercentages"]["Quantitative"].values())

# Create grouped bars
bars1 = ax.bar(x - width/2, mixed_compliance, width, label='Mixed Methods (n=18)', color=colors["Mixed Methods"], 
              edgecolor='white', linewidth=1.2)
bars2 = ax.bar(x + width/2, quant_compliance, width, label='Quantitative (n=11)', color=colors["Quantitative"],
              edgecolor='white', linewidth=1.2)

# Add value labels on bars
for bar in bars1:
    height = bar.get_height()
    ax.text(
        bar.get_x() + bar.get_width()/2., height + 1,
        f'{height:.1f}%', ha='center', va='bottom', fontsize=11, fontweight='bold'
    )
    
for bar in bars2:
    height = bar.get_height()
    ax.text(
        bar.get_x() + bar.get_width()/2., height + 1,
        f'{height:.1f}%', ha='center', va='bottom', fontsize=11, fontweight='bold'
    )

# Add some text for labels, title and axes ticks
ax.set_ylabel('Compliance (%)', fontsize=16, labelpad=15)
#ax.set_title('MMAT Criteria Compliance by Study Type', fontsize=18, pad=20)
ax.set_xticks(x)
ax.set_xticklabels(criteria, rotation=30, ha='right')
ax.set_ylim(0, 110)
ax.legend(loc='upper center', ncol=2, fontsize=14)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('figures/criteria_compliance.png', dpi=300, bbox_inches='tight')
plt.close()

# Figure 4: Radar Chart for Methodological Quality Profile
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, polar=True)

# Number of variables
categories = list(data["criteriaLabels"].values())
N = len(categories)

# What will be the angle of each axis in the plot (divide the plot into equal parts)
angles = [n / float(N) * 2 * np.pi for n in range(N)]
angles += angles[:1]  # Close the loop

# Set the labels
ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories, fontsize=12)

# Draw the polygon for Mixed Methods
values = list(data["compliancePercentages"]["Mixed Methods"].values())
values += values[:1]  # Close the loop
ax.plot(angles, values, linewidth=2, linestyle='solid', label="Mixed Methods", color=colors["Mixed Methods"])
ax.fill(angles, values, color=colors["Mixed Methods"], alpha=0.25)

# Draw the polygon for Quantitative
values = list(data["compliancePercentages"]["Quantitative"].values())
values += values[:1]  # Close the loop
ax.plot(angles, values, linewidth=2, linestyle='solid', label="Quantitative", color=colors["Quantitative"])
ax.fill(angles, values, color=colors["Quantitative"], alpha=0.25)

# Add legend
plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1), fontsize=14)

# Add percentage labels at each axis
for i, angle in enumerate(angles[:-1]):
    for study_type in data["compliancePercentages"].keys():
        value = data["compliancePercentages"][study_type][f"Q{i+1}"]
        if value > 0:
            offset = 10 if study_type == "Mixed Methods" else -10
            ha = 'left' if angle < np.pi else 'right'
            color = colors[study_type]
            ax.annotate(
                f'{value:.0f}%',
                xy=(angle, value+5),
                xytext=(angle+np.radians(offset), value+5),
                ha=ha,
                color=color,
                fontsize=10,
                fontweight='bold'
            )

# Fix axis to go from 0 to 100
ax.set_ylim(0, 100)
ax.set_title('Methodological Quality Profile by Study Type', fontsize=18, y=1.08)

# Set the y-axis labels
ax.set_yticks([20, 40, 60, 80, 100])
ax.set_yticklabels(['20%', '40%', '60%', '80%', '100%'], fontsize=10)
plt.tight_layout()
plt.savefig('figures/radar_profile.png', dpi=300, bbox_inches='tight')
plt.close()

# Figure 5: MMAT Score Distribution
fig, ax = plt.subplots(figsize=(12, 7))

# Combine all score distributions
all_scores = {'0': 0, '20': 0, '40': 0, '60': 0, '80': 0, '100': 0}
for study_type, scores in data["scoreDistribution"].items():
    for score, count in scores.items():
        all_scores[score] = all_scores.get(score, 0) + count

# Prepare data for plotting
score_labels = [f'{score} points' for score in all_scores.keys()]
score_counts = list(all_scores.values())

# Define colors based on score
score_colors = ['#95a5a6', '#e67e22', '#e67e22', '#f1c40f', '#2ecc71', '#27ae60']  # Grey, Orange, Yellow, Green, Dark Green

# Create bars
bars = ax.bar(score_labels, score_counts, color=score_colors, edgecolor='white', linewidth=1.5)

# Add value labels
for bar in bars:
    height = bar.get_height()
    ax.text(
        bar.get_x() + bar.get_width()/2., height + 0.1,
        str(int(height)), ha='center', va='bottom', fontsize=12, fontweight='bold'
    )

# Add percentage labels
total_studies = sum(score_counts)
for i, bar in enumerate(bars):
    height = bar.get_height()
    if height > 0:
        percentage = (height / total_studies) * 100
        ax.text(
            bar.get_x() + bar.get_width()/2., height/2,
            f'{percentage:.1f}%', ha='center', va='center', 
            color='white', fontsize=12, fontweight='bold'
        )

ax.set_ylabel('Number of Studies', fontsize=16, labelpad=15)
ax.set_xlabel('MMAT Score', fontsize=16, labelpad=15)
ax.set_title('Distribution of MMAT Scores Across All Studies (n=29)', fontsize=18, pad=20)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('figures/score_distribution.png', dpi=300, bbox_inches='tight')
plt.close()

# Figure 6: Score Distribution by Study Type
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Mixed Methods Score Distribution
mixed_scores = data["scoreDistribution"]["Mixed Methods"]
mixed_labels = [f'{score} points' for score in mixed_scores.keys()]
mixed_counts = list(mixed_scores.values())
mixed_colors = ['#f1c40f', '#2ecc71', '#27ae60']  # For 60, 80, 100

# Create pie chart for Mixed Methods
wedges1, texts1, autotexts1 = ax1.pie(
    mixed_counts, 
    labels=mixed_labels,
    autopct='%1.1f%%',
    startangle=90,
    colors=mixed_colors,
    explode=[0.05]*len(mixed_counts),
    shadow=True,
    wedgeprops={'edgecolor': 'white', 'linewidth': 1.5}
)

# Style the text
for text in texts1:
    text.set_fontsize(12)
for autotext in autotexts1:
    autotext.set_fontsize(11)
    autotext.set_fontweight('bold')
    autotext.set_color('white')

ax1.set_title('Mixed Methods Scores (n=18)', fontsize=16, pad=20)

# Quantitative Score Distribution
quant_scores = data["scoreDistribution"]["Quantitative"]
quant_labels = [f'{score} points' for score in quant_scores.keys()]
quant_counts = list(quant_scores.values())
quant_colors = ['#e67e22', '#f1c40f', '#2ecc71', '#27ae60']  # For 40, 60, 80, 100

# Create pie chart for Quantitative
wedges2, texts2, autotexts2 = ax2.pie(
    quant_counts, 
    labels=quant_labels,
    autopct='%1.1f%%',
    startangle=90,
    colors=quant_colors,
    explode=[0.05]*len(quant_counts),
    shadow=True,
    wedgeprops={'edgecolor': 'white', 'linewidth': 1.5}
)

# Style the text
for text in texts2:
    text.set_fontsize(12)
for autotext in autotexts2:
    autotext.set_fontsize(11)
    autotext.set_fontweight('bold')
    autotext.set_color('white')

ax2.set_title('Quantitative Scores (n=11)', fontsize=16, pad=20)

plt.tight_layout()
plt.savefig('figures/score_by_type.png', dpi=300, bbox_inches='tight')
plt.close()

# Create a full report with all figures
fig = plt.figure(figsize=(15, 20))
plt.suptitle('MMAT Analysis - May 2025', fontsize=24, y=0.98)

# Add subtitle
plt.figtext(0.5, 0.955, 'Methodological Quality Assessment of 29 Educational Technology Studies', 
           ha='center', fontsize=16, style='italic')

# Load all the images
img1 = plt.imread('figures/study_distribution.png')
img2 = plt.imread('figures/average_scores.png')
img3 = plt.imread('figures/criteria_compliance.png')
img4 = plt.imread('figures/radar_profile.png')
img5 = plt.imread('figures/score_distribution.png')
img6 = plt.imread('figures/score_by_type.png')

# Create a grid of subplots
grid = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.1)

# Add each image to a subplot
ax1 = fig.add_subplot(grid[0, 0])
ax1.imshow(img1)
ax1.axis('off')
ax1.set_title('1. Distribution of Studies by Methodology', fontsize=14, pad=10)

ax2 = fig.add_subplot(grid[0, 1])
ax2.imshow(img2)
ax2.axis('off')
ax2.set_title('2. Average MMAT Score by Study Type', fontsize=14, pad=10)

ax3 = fig.add_subplot(grid[1, :])
ax3.imshow(img3)
ax3.axis('off')
ax3.set_title('3. MMAT Criteria Compliance by Study Type', fontsize=14, pad=10)

ax4 = fig.add_subplot(grid[2, 0])
ax4.imshow(img4)
ax4.axis('off')
ax4.set_title('4. Methodological Quality Profile', fontsize=14, pad=10)

ax5 = fig.add_subplot(grid[2, 1])
ax5.imshow(img5)
ax5.axis('off')
ax5.set_title('5. Distribution of MMAT Scores', fontsize=14, pad=10)

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.savefig('figures/mmat_full_report.png', dpi=300, bbox_inches='tight')
plt.close()

print("All figures have been generated in the 'figures' directory:")
print("1. study_distribution.png")
print("2. average_scores.png")
print("3. criteria_compliance.png")
print("4. radar_profile.png")
print("5. score_distribution.png")
print("6. score_by_type.png")
print("7. mmat_full_report.png")

C:\Users\sefra\AppData\Local\Temp\ipykernel_42212\1152189985.py:404: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 1, 0.95])


All figures have been generated in the 'figures' directory:
1. study_distribution.png
2. average_scores.png
3. criteria_compliance.png
4. radar_profile.png
5. score_distribution.png
6. score_by_type.png
7. mmat_full_report.png
